## APP PASSWORD

In [1]:
APP_PASSWORD = 'hksx zxjb wztw ncvp'

## Ebay

In [5]:
import imaplib
import email
from email.header import decode_header

# IMAP server settings for Gmail
IMAP_SERVER = 'imap.gmail.com'
USERNAME = 'dateng2016@gmail.com'
PASSWORD = APP_PASSWORD

# Connect to the IMAP server
mail = imaplib.IMAP4_SSL(IMAP_SERVER)
mail.login(USERNAME, PASSWORD)

# Select the mailbox (e.g., 'INBOX')
mail.select('INBOX')

# Search for emails based on specific criteria (e.g., subject, sender, date)
search_criteria = '(SUBJECT "Da, your order is confirmed")'
result, data = mail.search(None, search_criteria)

# Get the email IDs
email_ids = data[0].split()

# Retrieve the most recent email (assuming the search returns emails sorted by date)
latest_email_id = email_ids[-1]

# Fetch the email data (headers and body)
result, email_data = mail.fetch(latest_email_id, '(RFC822)')
raw_email = email_data[0][1]

# Parse the raw email data
msg = email.message_from_bytes(raw_email)

# Extract email information (e.g., sender, subject)
sender = msg['From']
subject = decode_header(msg['Subject'])[0][0]

# Print email information
print("Sender:", sender)
print("Subject:", subject)

# You can access email body and attachments similarly
# body = msg.get_payload()

# Function to get the email body
def get_email_body(msg):
    if msg.is_multipart():
        # If the message is multipart, iterate over each part
        for part in msg.walk():
            content_type = part.get_content_type()
            if content_type == "text/plain" or content_type == "text/html":
                # If the part is plain text or HTML, return its payload
                return part.get_payload(decode=True).decode()
    else:
        # If the message is not multipart, return its payload
        return msg.get_payload(decode=True).decode()

# Example usage:
email_body = get_email_body(msg)
# print("Email Body:", email_body)


# Close the connection
mail.close()
mail.logout()

Sender: eBay <ebay@ebay.com>
Subject: Da, your order is confirmed


('BYE', [b'LOGOUT Requested'])

In [4]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(email_body, 'html.parser')

name_tag = soup.find('a', class_='title')
price_tags = soup.find_all('td', class_="item")
print(f'Merchant name: {sender.split(" ")[0]}')
print()
print('Item: ', name_tag.string)
print('Price: ',price_tags[-1].string)
print('Date: ', msg['Date'])

Merchant name: eBay

Item:  
3.5mm Jack To 2 Rca Stereo Audio Cable Adapter 6ft 10...

Price:  
$5.07

Date:  Tue, 13 Feb 2024 17:33:11 -0700


## Gmail API


In [28]:
import os.path
import os
import pickle
from bs4 import BeautifulSoup

# for encoding/decoding messages in base64
from base64 import urlsafe_b64decode, urlsafe_b64encode
# for dealing with attachement MIME types
import email
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from mimetypes import guess_type as guess_mime_type

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]



def start_service():
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json", SCOPES
            )
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    try:
    # Call the Gmail API
        service = build("gmail", "v1", credentials=creds)
        results = service.users().labels().list(userId="me").execute()
        labels = results.get("labels", [])

        if not labels:
            print("No labels found.")

        # print("Labels:")
        for label in labels:
        #   print(label["name"])
            pass
        return service

    except HttpError as error:
    # TODO(developer) - Handle errors from gmail API.
        print(f"An error occurred: {error}")


def search_messages(service, query):
    result = service.users().messages().list(userId='me',q=query).execute()
    messages = [ ]
    if 'messages' in result:
        messages.extend(result['messages'])
    while 'nextPageToken' in result:
        page_token = result['nextPageToken']
        result = service.users().messages().list(userId='me',q=query, pageToken=page_token).execute()
        if 'messages' in result:
            messages.extend(result['messages'])
    return messages

def parse_parts(service, parts, folder_name, result_list, file_count):
    """
    Utility function that parses the content of an email partition
    """
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    if parts:
        for part in parts:
            filename = part.get("filename")
            mimeType = part.get("mimeType")
            body = part.get("body")
            data = body.get("data")
            file_size = body.get("size")
            part_headers = part.get("headers")

            binary_data = urlsafe_b64decode(data)
            if part.get("parts"):
                # recursively call this function when we see that a part
                # has parts inside
                file_count[0] += 1
                parse_parts(service, part.get("parts"), folder_name, result_list, file_count)
            if mimeType == "text/plain":
                # if the email part is text plain
                if data:
                    text = binary_data.decode()
                    print(text)
            elif mimeType == "text/html":
                # if the email part is an HTML content
                # save the HTML file and optionally open it in the browser
                if not filename:
                    file_count[0] += 1
                    filename = "index"+str(file_count[0])+".html"
                filepath = os.path.join(folder_name, filename)
                print("Saving HTML to", filepath)
                with open(filepath, "wb") as f:
                    f.write(binary_data)
                # print(binary_data.decode())
                result_list.append(binary_data.decode())

        
def parse_ebay():
    merchant_name = 'ebay'
    




if __name__ == "__main__":
    service = start_service()
    search_query = 'from:ebay@ebay.com subject:"Da, your order is confirmed"'
    # Call the Gmail API to search for messages
    messages = service.users().messages().list(userId='me', q=search_query).execute()
    html_list = []



    # Check if any messages match the search criteria
    file_count = [0]
    if 'messages' in messages:
        for message in messages['messages']:
            # Get the message details
            msg_id = message['id']
            # print((msg_id))
            msg = service.users().messages().get(userId='me', id=msg_id).execute()
            print(msg['payload']['headers'][24]['value'])
            parse_parts(service, msg['payload']['parts'], 'ReceiptFolder', html_list,file_count)

            # Print the snippet (preview) of the message
            # print('Message snippet:', msg['snippet'])
            # print(msg['payload'].keys())
            # parts = msg['payload']['parts']
            # print(parts[0].keys())
            # print((parts[0]['body']))
            # Print the full message (optional)
            # print('Full message:', msg)
    else:
        print('No messages found matching the search criteria.')
    print(len(html_list))
    print(type(email.message_from_string(html_list[0])))



todo:11400:eBayStarship
Saving HTML to ReceiptFolder/index1.html
todo:11400:eBayStarship
Saving HTML to ReceiptFolder/index2.html
2


TypeError: initial_value must be str or None, not list

In [25]:
message = service.users().messages().get(userId='me', id='18da50829cbae0aa', format='metadata').execute()

# Extract the internal date of the message
internal_date = int(message['internalDate']) / 1000  # Convert milliseconds to seconds
print("Date of the email:", internal_date)
from datetime import datetime

# Convert the timestamp to a datetime object
email_date = datetime.utcfromtimestamp(internal_date)

# Format the datetime object as a string
formatted_email_date = email_date.strftime('%Y-%m-%d %H:%M:%S')

print("Date of the email:", formatted_email_date)

Date of the email: 1707870791.0
Date of the email: 2024-02-14 00:33:11


In [11]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_list[0], 'html.parser')

name_tag = soup.find('a', class_='title')
price_tags = soup.find_all('td', class_="item")
# print(f'Merchant name: {sender.split(" ")[0]}')
print()
print('Item: ', name_tag.string)
print('Price: ',price_tags[-1].string)
# print('Date: ', msg['Date'])


Item:  
3.5mm Jack To 2 Rca Stereo Audio Cable Adapter 6ft 10...

Price:  
$5.07

